# Test Greykite

### Imports

In [47]:
import pandas as pd
from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector
from greykite.algo.forecast.silverkite.constants.silverkite_holiday import SilverkiteHoliday
from greykite.algo.forecast.silverkite.constants.silverkite_seasonality import SilverkiteSeasonalityEnum
from greykite.algo.forecast.silverkite.forecast_simple_silverkite_helper import cols_interact
from greykite.common import constants as cst
from greykite.common.features.timeseries_features import build_time_features_df
from greykite.common.features.timeseries_features import convert_date_to_continuous_time
from greykite.framework.benchmark.data_loader_ts import DataLoaderTS
from greykite.framework.templates.autogen.forecast_config import EvaluationPeriodParam
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries

In [48]:
retail_csv = pd.read_csv('/Users/fernandoazuaje/Documents/work/3mit/AI/retail_sales_Dataset.csv')
df = pd.DataFrame(retail_csv)
df['Date'] = pd.to_datetime(df['Date'])


In [49]:
df = df.query('`Product Category` == "Clothing"')
    
df = df.groupby(['Date']).agg({'Quantity': 'sum'}).reset_index()

df

,Date,Quantity
0,2023-01-01,7
1,2023-01-02,6
2,2023-01-04,3
3,2023-01-06,1
4,2023-01-07,3
...,...,...
227,2023-12-21,1
228,2023-12-23,4
229,2023-12-24,3
230,2023-12-26,5


In [50]:
# No es un dataframe pero pareceira un dataframe5
ts = UnivariateTimeSeries()
ts.load_data(
    df=df,
    time_col="Date",
    value_col="Quantity",
    freq="D"
    # anomaly_info=anomaly_info,
    # regressor_cols=["sale_price"]
)

metadata = MetadataParam(
        time_col="Date",  # name of the time column
        value_col="Quantity",  # name of the value column
        freq="D"  # "H" for hourly, "D" for daily, "W" for weekly, etc.
    )

/Users/fernandoazuaje/Documents/work/3mit/AI/3mit_ai_testing/greykite-env/lib/python3.10/site-packages/greykite/common/time_properties.py:419: UserWarning:

`train_end_date` is not provided, or Quantity column of the provided time series contains null values at the end, or the input `train_end_date` is beyond the last timestamp available. Setting `train_end_date` to the last timestamp with a non-null value (2023-12-28 00:00:00).



In [51]:
forecaster = Forecaster()

result = forecaster.run_forecast_config(
            df=df,
            config=ForecastConfig(
                model_template=ModelTemplateEnum.SILVERKITE.name,
                forecast_horizon=72, 
                coverage=0.90,
                metadata_param=metadata,
                # model_components_param=model_components,
                # evaluation_period_param=evaluation_period 
            )
        )

/Users/fernandoazuaje/Documents/work/3mit/AI/3mit_ai_testing/greykite-env/lib/python3.10/site-packages/greykite/common/time_properties.py:419: UserWarning:

`train_end_date` is not provided, or Quantity column of the provided time series contains null values at the end, or the input `train_end_date` is beyond the last timestamp available. Setting `train_end_date` to the last timestamp with a non-null value (2023-12-28 00:00:00).

/Users/fernandoazuaje/Documents/work/3mit/AI/3mit_ai_testing/greykite-env/lib/python3.10/site-packages/greykite/common/time_properties.py:419: UserWarning:

`train_end_date` is not provided, or Quantity column of the provided time series contains null values at the end, or the input `train_end_date` is beyond the last timestamp available. Setting `train_end_date` to the last timestamp with a non-null value (2023-12-28 00:00:00).

/Users/fernandoazuaje/Documents/work/3mit/AI/3mit_ai_testing/greykite-env/lib/python3.10/site-packages/greykite/sklearn/transform/nu

Fitting 2 folds for each of 1 candidates, totalling 2 fits


/Users/fernandoazuaje/Documents/work/3mit/AI/3mit_ai_testing/greykite-env/lib/python3.10/site-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 50.00% of one input.

/Users/fernandoazuaje/Documents/work/3mit/AI/3mit_ai_testing/greykite-env/lib/python3.10/site-packages/greykite/common/evaluation.py:153: UserWarning:

36 value(s) in y_true were NA or infinite and are omitted in error calc.

/Users/fernandoazuaje/Documents/work/3mit/AI/3mit_ai_testing/greykite-env/lib/python3.10/site-packages/greykite/sklearn/transform/null_transformer.py:177: RuntimeWarning:

Input data has many null values. Missing 50.00% of one input.

/Users/fernandoazuaje/Documents/work/3mit/AI/3mit_ai_testing/greykite-env/lib/python3.10/site-packages/greykite/common/evaluation.py:153: UserWarning:

36 value(s) in y_true were NA or infinite and are omitted in error calc.

/Users/fernandoazuaje/Documents/work/3mit/AI/3mit_ai_testing/greykite-env/lib/

In [52]:
result.forecast.plot()